In [1]:
# Import all the necessary packages
import glob
import earthaccess
import numpy as np
import rasterio
import rioxarray as rio
import xarray as xr
from pathlib import Path
from rasterio.enums import Resampling

In [2]:
### Use this section if you want to search for data you don't have downloaded ###
auth = earthaccess.login(persist=True)

# Uncomment the parameters to include them in your search
results = earthaccess.search_data(
    short_name = "PACE_OCI_L3M_LANDVI",
    temporal = ("2024-07-01", "2024-8-01"),                 # Change this for time span
    #bounding_box = (w, s, e, n)                            # Change this for extent
    granule_name="*8D*0p1*",                                # Specifics of filename
)

# This will display each file that matched your search
#for item in results:
#    display(item)

# This will download everyfile that matched your results in a folder called "data"
# This step might take a while also!
#paths = earthaccess.download(results, local_path="data")

Granules found: 5


In [3]:
### If you already have the data downloaded, use this instead ###

# Replace with the path to the folder that contains your files 
# But keep the /*.nc, as that will grab only the netcdfs in that folder
paths = sorted(glob.glob("path/to/your/data/*.nc"))
paths

['data/PACE_OCI.20240711_20240718.L3m.8D.LANDVI.V3_0.0p1deg.nc',
 'data/PACE_OCI.20240719_20240726.L3m.8D.LANDVI.V3_0.0p1deg.nc',
 'data/PACE_OCI.20240727_20240803.L3m.8D.LANDVI.V3_0.0p1deg.nc']

The `xr.open_mfdataset()` utility is what we'll use to concatenate along a new dimension, which I called `time` but you can call anything you like. 

In [4]:
# Open the datasets together with a time dimension
dataset = xr.open_mfdataset(
    paths,
    combine="nested",
    concat_dim="time",
).drop_vars("palette")

# And print it out to see your new combined dataset
dataset

# Note for fun: you can also compute across the time dimension, e.g. the average
# ds_mean = dataset.mean("time")

<xarray.Dataset> Size: 778MB
Dimensions:  (time: 3, lat: 1800, lon: 3600)
Coordinates:
  * lat      (lat) float32 7kB 89.95 89.85 89.75 89.65 ... -89.75 -89.85 -89.95
  * lon      (lon) float32 14kB -179.9 -179.9 -179.8 ... 179.8 179.9 180.0
Dimensions without coordinates: time
Data variables:
    ndvi     (time, lat, lon) float32 78MB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    evi      (time, lat, lon) float32 78MB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    ndwi     (time, lat, lon) float32 78MB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    ndii     (time, lat, lon) float32 78MB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    cci      (time, lat, lon) float32 78MB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    ndsi     (time, lat, lon) float32 78MB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    pri      (time, lat, lon) float32 78MB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    cire     (time, lat, lon) float32 78MB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    car      (time, lat, lon) float32 78MB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    mari     (time, lat, lon) float32 78MB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
Attributes: (12/62)
    product_name:                      PACE_OCI.20240711_20240718.L3m.8D.LAND...
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    cdm_data_type:                     grid
    identifier_product_doi_authority:  http://dx.doi.org
    identifier_product_doi:            10.5067/PACE/OCI/L3M/LANDVI/3.0
    data_bins:                         1350123
    data_minimum:                      -71.82708
    data_maximum:                      35.656807

Now we can download the datasets as .tifs. With this method, you can only do one VI at a time since `rioxarray` doesn't support conversion to 4D GeoTIFFs (which aren't the ideal way to handle data anyway). 

That is also why, if you wanted to do this for SFREFL, you'd have to first choose which wavelength to export. 

In [5]:
# First set the CRS
dataset = dataset.rio.write_crs("epsg:4326")

# Change CIRE to any of the VI names in the dataset
dataset["cire"].rio.to_raster("path/to/your/file.tif", driver="COG")

And you're done!